# NPC + Bullet

Как создать NPC, который будет стреляться?

1. Создать класс для пули (Bullet)
2. Базовые функции

Давайте для начала преобразуем наш класс `NPC`, чтобы тот выступал в качестве класса-родителя для всех остальных `npc`

Далее мы просто будем наследоваться от этого класса, это позволит нам быстро реализовывать задуманные `npc` и гарантировать целостность программы 

Создаем папку `npc` туда положим `npc.py` и `npc_control.py`. У нас поменяются импорты, мы хотим в `main.py` импортировать папку npc целиком:

```python
#main.py
from npc import *
```

*Напоминание*: запустится файл `__init__.py`, который лежит в импортируемой папке

То есть наши изменения выглядят следующим образом:

$\color{red}{\text{- npc.py}}$

$\color{green}{\text{+ npc}}$ \
$\hookrightarrow\color{green}{\text{+ npc.py}}$ \
$\hookrightarrow\color{green}{\text{+ npc\_control.py}}$ \
$\hookrightarrow\color{green}{\text{+ \_\_init\_\_.py}}$

Далее создадим папки для каждого типа `npc` в отдельности. У нас уже реализован "бегун" (`runner`), также сегодня реализуем "шутер" (shooter):

${\color{silver}\text{npc}}$ \
$\hookrightarrow\color{green}{\text{+ runner}}$ \
$\qquad\hookrightarrow\color{green}{\text{+ runner.py}}$ \
$\qquad\hookrightarrow\color{green}{\text{+ \_\_init\_\_.py}}$ \
$\hookrightarrow\color{green}{\text{+ shooter}}$ \
$\qquad\hookrightarrow\color{green}{\text{+ shooter.py}}$ \
$\qquad\hookrightarrow\color{green}{\text{+ bullet.py}}$ \
$\qquad\hookrightarrow\color{green}{\text{+ \_\_init\_\_.py}}$

В `npc_control` будем импортировать папки с нужными нам `npc`, в `__init__.py` будем настраивать логику импортов внутри модуля:

```python
#npc_control.py
from .runner import *
from .shooter import *


class NpcControl:
    ...
```

Давайте теперь оставим в классе `NPC` только те методы и аргументы, которые будут общими для каждого типа `npc`. В нашем случае это все функции, кроме `move` (так как двигаться будет только `Runner`), а также функцию `logic` мы будем переопределять для каждого типа `npc`

 Выглядеть все сказанное будет следующим образом:

In [ ]:
import pygame as pg
import math
from settings import *


class NPC:
    def __init__(self, game, pos, color):
        self.x, self.y = pos
        self.color = color
        self.game = game
        self.player = game.player
        self.theta = 0

    @property
    def map_pos(self):
        return int(self.x), int(self.y)

    def draw(self):
        pg.draw.circle(self.game.screen, self.color,
                       (100 * self.x, 100 * self.y), 15)
        pg.draw.line(self.game.screen, self.color, (self.x * 100 - 14.5 * math.sin(self.theta), self.y * 100 + 14.5 * math.cos(self.theta)),
                     (self.x * 100 + 35 * math.cos(self.theta),
                     self.y * 100 + 35 * math.sin(self.theta)), 2)
        pg.draw.line(self.game.screen, self.color, (self.x * 100 + 14.5 * math.sin(self.theta), self.y * 100 - 14.5 * math.cos(self.theta)),
                     (self.x * 100 + 35 * math.cos(self.theta),
                     self.y * 100 + 35 * math.sin(self.theta)), 2)

    def look(self):
        dx = self.player.x - self.x
        dy = self.player.y - self.y
        self.theta = math.atan2(dy, dx)



    def check_wall(self, x, y):
        return (x, y) not in self.game.map.world_map

    def check_wall_collision(self, dx, dy):
        if self.check_wall(int(self.x + dx), int(self.y)):
            self.x += dx
        if self.check_wall(int(self.x), int(self.y + dy)):
            self.y += dy

    def ray_cast_player_npc(self):
        if self.game.player.map_pos == self.map_pos:
            return True

        wall_dist_v, wall_dist_h = 0, 0
        player_dist_v, player_dist_h = 0, 0

        ox, oy = self.game.player.pos
        x_map, y_map = self.game.player.map_pos

        ray_angle = math.pi + self.theta

        sin_a = math.sin(ray_angle)
        cos_a = math.cos(ray_angle)

        # horizontals
        y_hor, dy = (y_map + 1, 1) if sin_a > 0 else (y_map - 1e-6, -1)

        depth_hor = (y_hor - oy) / sin_a
        x_hor = ox + depth_hor * cos_a

        delta_depth = dy / sin_a
        dx = delta_depth * cos_a

        for i in range(MAX_DEPTH):
            tile_hor = int(x_hor), int(y_hor)
            if tile_hor == self.map_pos:
                player_dist_h = depth_hor
                break
            if tile_hor in self.game.map.world_map:
                wall_dist_h = depth_hor
                break
            x_hor += dx
            y_hor += dy
            depth_hor += delta_depth

        # verticals
        x_vert, dx = (x_map + 1, 1) if cos_a > 0 else (x_map - 1e-6, -1)

        depth_vert = (x_vert - ox) / cos_a
        y_vert = oy + depth_vert * sin_a

        delta_depth = dx / cos_a
        dy = delta_depth * sin_a

        for _ in range(MAX_DEPTH):
            tile_vert = int(x_vert), int(y_vert)
            if tile_vert == self.map_pos:
                player_dist_v = depth_vert
                break
            if tile_vert in self.game.map.world_map:
                wall_dist_v = depth_vert
                break
            x_vert += dx
            y_vert += dy
            depth_vert += delta_depth

        player_dist = max(player_dist_v, player_dist_h)
        wall_dist = max(wall_dist_v, wall_dist_h)

        if 0 < player_dist < wall_dist or not wall_dist:
            return True
        return False

    def logic(self):
        pass

    def update(self):
        self.look()
        self.logic()


Как видите, функция `logic` (логика нашего персонажа) не реализованна, её мы будет реализовывать в дочерних классах. При этом нетрудно заметить, что по факту для реализации некоторого типа `npc` нам только надо реализовать функцию `logic` и нужные для неё функции. То есть для `Runner` -- `move`, для `Shooter` -- `fire` и т.д. В этом и сила наследования

## Runner

Теперь создадим класс `Runner`, отнаследуемся от `NPC`, настроим функцию `logic` и добавим ранее написанную функцию `move`:

In [ ]:
#runner.py
from ..npc import *


class Runner(NPC):
    def __init__(self, game, pos, color=DEFAULT_COLOR, speed=SPEED_NPC):
        super().__init__(game=game, pos=pos, color=color)
        self.speed = speed

    def move(self):
        next_pos = self.game.path_finding.get_path(
            self.map_pos, self.game.player.map_pos)
        next_x, next_y = next_pos

        angle = math.atan2(next_y + 0.5 - self.y, next_x + 0.5 - self.x)
        dx = math.cos(angle) * self.speed * self.game.delta_time
        dy = math.sin(angle) * self.speed * self.game.delta_time

        if next_pos not in self.game.npc_control.npc_positions:
            self.check_wall_collision(dx, dy)

    def logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.move()


Как видите заново функцию `__init__` я тоже не хочу реализовывать, потому что она в целом похожа на ту, что в `NPC`. Поэтому я хочу запустить конструтор родителя, а затем добавить аргументы, которого у него нет, в данном случае это агрумент `speed`:

```python

def __init__(self, game, pos, color=DEFAULT_COLOR, speed=SPEED_NPC):
    super().__init__(game=game, pos=pos, color=color)
    self.speed = speed
```

Теперь в `NpcControl` вместо того, чтобы добавлять экземпляры класса `NPC` добавляем `Runner`:

```python
from .runner import *
from .shooter import *


class NpcControl:
    ...

    def add_npc(self):
        self.npc_list = [
            Runner(game=self.game, pos=(11.5, 7.5), color='blue'),
            Runner(game=self.game, pos=(5, 7.5), color='blue'),
        ]
    ...
```

## Shooter

Логика такая же, что при создании `Runner`: 
1. Реализуем функцию `logic`
2. Пишем вспомагательные для неё методы

In [ ]:
#shooter.py
from ..npc import *
from .bullet import *

class Shooter(NPC):
    def __init__(self, game, pos, color=DEFAULT_COLOR):
        super().__init__(game=game, pos=pos, color=color)
        self.bullets = []

    def logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.fire()

    def fire(self):
        Bullet(*self.bullet_params, theta=self.theta)


Осталось реализовать `Bullet`, реализовав стандартные методы:
1. `__init__`
2. `draw`
3. `update`
4. `logic` (+ вспомогательные для неё функции, в нашем случае это движение пули, то есть `move`)

In [ ]:
#bullet.py
import pygame as pg
import math
from settings import *

class Bullet():
    def __init__(self, game, pos, color, speed, theta):
        self.x, self.y = pos
        self.game = game
        self.color = color
        self.theta = theta
        self.speed = speed
        self.strike = False
        
    def kill(self):
        len_to_player = ((self.game.player.x - self.x) ** 2 + (self.game.player.y - self.y) ** 2)
        if len_to_player < BULLET_KILL_LEN:
            self.strike = True

    def move(self):
        dx = math.cos(self.theta) * self.speed * self.game.delta_time
        dy = math.sin(self.theta) * self.speed * self.game.delta_time
        self.check_wall_collision(dx, dy)

    def draw(self):
        pg.draw.circle(self.game.screen, self.color,
                       (100 * self.x, 100 * self.y),  5)

    def check_wall(self, x, y):
        if (x, y) not in self.game.map.world_map:
            return True
        else:
            self.strike = True
            return False

    def check_wall_collision(self, dx, dy):
        if self.check_wall(int(self.x + dx), int(self.y)):
            self.x += dx
        if self.check_wall(int(self.x), int(self.y + dy)):
            self.y += dy
            
    def update(self):
        self.move()
        self.kill()


Мы создаем пули, когда видим игрока, осталось где-то запустить функции `draw` и `update` самой пули. Так как пулю выпускает `Shooter`, то давайте обновим `draw` и `update` его, запустив сначала метод его родителя, а потом отрисовку всех пуль, которые он выстрелил: 

In [ ]:
#shooter.py
from ..npc import *
from .bullet import *

class Shooter(NPC):
    def __init__(self, game, pos, color=DEFAULT_COLOR):
        super().__init__(game=game, pos=pos, color=color)
        self.bullets = []

    def logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.fire()

    def fire(self):
        Bullet(*self.bullet_params, theta=self.theta)
        
        
    def draw(self):
        super().draw()
        for bullet in self.bullets:
            bullet.draw()

    def update(self):
        super().update()

        bullets = []

        for bullet in self.bullets:
            bullet.update()
            if not bullet.strike:
                bullets.append(bullet)

        self.bullets = bullets


Теперь, чтобы мы могли лучше настраивать `Shooter`, добавим таймер и ограничение на количество пуль

In [ ]:
from ..npc import *
from .bullet import *
import pygame as pg


class Shooter(NPC):
    def __init__(self, game, pos, color=DEFAULT_COLOR, ttl=TTL, bullet_cnt=BULLET_CNT, bullet_speed=BULLET_SPEED):
        super().__init__(game=game, pos=pos, color=color)
        self.bullet_cnt = bullet_cnt
        self.bullet_params = [game, pos, color, bullet_speed]
        self.bullets = []

        self.timer_activate = False
        self.ttl = ttl
        self.time = pg.time.get_ticks() - ttl

    def logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.fire()

    @property
    def timer(self):
        if self.time + self.delay <= pg.time.get_ticks():
            self.time = pg.time.get_ticks()
            return True
        else:
            return False

    @timer.setter
    def timer(self, delay):
        self.delay = delay

    def toggle_timer(self):
        if self.timer_activate:
            self.timer = 0
        else:
            self.timer = self.ttl

        self.timer_activate = not self.timer_activate

    def fire(self):
        if len(self.bullets) < self.bullet_cnt:
            if not self.timer_activate:
                self.toggle_timer()
            if self.timer:
                self.bullets.append(
                    Bullet(*self.bullet_params, theta=self.theta))
        else:
            if self.timer_activate:
                self.toggle_timer()

    def draw(self):
        super().draw()
        for bullet in self.bullets:
            bullet.draw()

    def update(self):
        super().update()

        bullets = []

        for bullet in self.bullets:
            bullet.update()
            if not bullet.strike:
                bullets.append(bullet)

        self.bullets = bullets


Далее просто в `NpcControl` добавляем `Shooter`:

```python
#npc_control.py
from .runner import *
from .shooter import *


class NpcControl:
    ...

    def add_npc(self):
        self.npc_list = [
            Runner(game=self.game, pos=(11.5, 7.5), color='blue'),
            Shooter(game=self.game, pos=(6.5, 7.5), color='yellow', ttl=20, bullet_cnt=1000),
            Shooter(game=self.game, pos=(8, 3), ttl=250),
            Runner(game=self.game, pos=(5, 7.5), color='blue'),
            Shooter(game=self.game, pos=(15, 7.5), color='yellow', ttl=20, bullet_cnt=1000),
            Shooter(game=self.game, pos=(10, 5), ttl=20, bullet_cnt=1000),
        ]
    ...

```